In [ ]:
from bs4 import BeautifulSoup
import requests, json, re, os, time, pdb, concurrent.futures, io, random
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def download_image(url, filename):
    time.sleep(random.uniform(5, 6))
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        out = open(filename, 'wb')
        out.write(response.content)
        out.close()
        print(f"Image {filename[7:]} saved successfully")

In [ ]:
def WGA_Artists(url):
    #url = 'https://www.wga.hu/support/mobile/artists.html'
    alph = []
    artists = []
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    items = soup.find_all('div', class_ = 'artist1')
    for item in items:
        art_block = item.find_all("a")
        for block in art_block:
            href = block.get('href')
            alph.append(f"https://www.wga.hu{href}")
    for letter in alph:
        response = requests.get(letter)
        soup = BeautifulSoup(response.text, 'html.parser')
        items = soup.find_all('li')
        for item in items:
            href = item.find('a').get('href')
            artists.append(f"{letter[:-10]}{href}")
    return artists

In [ ]:
def images_WGA(url, metadata):
    #url = "https://www.wga.hu/html_m/n/nazari/index.html"
    response = requests.get(url)
    images_map = {}
    soup = BeautifulSoup(response.text, 'html.parser')

    #author = soup.find("div", class_ = "STYLE").get_text()
    table = soup.find("table", {"border" : "1", "width" : "97%", "bgcolor" : "#99cccc", "cellpadding" : "5"})

    if table is None:
        links = soup.find_all("ul")
        for link in links:
            try:
                href = link.find("li").find("a").get("href")
                new_page = f"{url[:-10]}{href}"
                response = requests.get(new_page)
                soup = BeautifulSoup(response.text, 'html.parser')
                table = soup.find("table", {"border" : "1", "width" : "97%", "bgcolor" : "#99cccc", "cellpadding" : "5"})
                if table is None:
                    links = soup.find_all("ul")
                    for link in links:
                        try:
                            href = link.find("li").find("a").get("href")
                            new_page = f"{url[:-10]}{href}"
                            response = requests.get(new_page)
                            soup = BeautifulSoup(response.text, 'html.parser')
                            table = soup.find("table", {"border" : "1", "width" : "97%", "bgcolor" : "#99cccc", "cellpadding" : "5"})
                            if table is None:
                                continue
                            else:
                                author = soup.find("div", class_ = "STYLE").get_text()
                                items = table.find_all("tr")
                                for item in items:
                                    try:
                                        info = item.find_all("td")
                                        data = info[1].get_text().replace("\n", "").split("\r")
                                        data[3] = author
                                        if data[2].split(",")[0] != "Photo" and data[2].split(",")[0] != "Marble" and data[2].split(",")[0] != "Engraving":
                                            try:
                                                metadata.loc[len(metadata)] = data
                                                href = item.find("td").find("a").get("href")
                                                image = f"https://www.wga.hu{href}"
                                                images_map[image] = f'images/{data[0]}.jpg'
                                            except ValueError:
                                                pass

                                    except (AttributeError, IndexError):
                                        pass
                        except AttributeError:
                            pass

                else:
                    author = soup.find("div", class_ = "STYLE").get_text()
                    items = table.find_all("tr")
                    for item in items:
                        try:
                            info = item.find_all("td")
                            data = info[1].get_text().replace("\n", "").split("\r")
                            data[3] = author
                            if data[2].split(",")[0] != "Photo" and data[2].split(",")[0] != "Marble" and data[2].split(",")[0] != "Engraving":
                                try:
                                    metadata.loc[len(metadata)] = data
                                    href = item.find("td").find("a").get("href")
                                    image = f"https://www.wga.hu{href}"
                                    images_map[image] = f'images/{data[0]}.jpg'
                                except ValueError:
                                    pass
                        except (AttributeError, IndexError):
                            pass
            except AttributeError:
                pass

    else:
        author = soup.find("div", class_ = "STYLE").get_text()
        items = table.find_all("tr")
        for item in items:
            try:
                info = item.find_all("td")
                data = info[1].get_text().replace("\n", "").split("\r")
                data[3] = author
                if data[2].split(",")[0] != "Photo" and data[2].split(",")[0] != "Marble" and data[2].split(",")[0] != "Engraving":
                    try:
                        metadata.loc[len(metadata)] = data
                        href = item.find("td").find("a").get("href")
                        image = f"https://www.wga.hu{href}"
                        images_map[image] = f'images/{data[0]}.jpg'
                    except ValueError:
                        pass
            except (AttributeError, IndexError):
                pass
    return images_map, metadata

In [ ]:
def CGFA_Artists(url):
    #url = 'http://www.sai.msu.su/cjackson/featured.htm'
    response = requests.get(url)
    artists = []
    soup = BeautifulSoup(response.text, 'html.parser')
    items = soup.find_all('blockquote')
    for item in items[1:]:
        art_block = item.find_all("p")
        for art in art_block:
            a = art.find_all('a')
            for _ in a:
                link = _.get('href')
                if link != None and link != '#index':
                    artists.append(f"http://www.sai.msu.su/cjackson/{link}")
    return artists

In [ ]:
def images_CGFA(url, metadata):
    #url = 'http://www.sai.msu.su/cjackson/aertsen/index.html'
    response = requests.get(url)
    imgs = []
    names = []
    soup = BeautifulSoup(response.text, 'html.parser')
    items = soup.find('blockquote')
    try:
        data = items.get_text()
        lines = data.strip().split('\n')
        #table = pd.DataFrame(columns=['Title', 'Year', 'Type', 'Artist'])
        #table = pd.DataFrame(columns=['Title', 'Year', 'Type', 'Artist'])        # Давай тут Text потом будем чистить. Где-то просто нету типа картины. Он смещен
        artist = url[31:-11]
        try:
            for line in lines:
                parts = line.split(', ')
                name = parts[0]
                names.append(f'images/{name.lstrip(" ")}.jpg')
                year = parts[1].split('.')[0]
                typ = parts[2]
                #text = parts[2:]
                metadata = metadata.append({'Title': name, 'Year': year, 'Type': typ, 'Artist': artist[0].upper() + artist[1:]}, ignore_index=True)
                #df = pd.DataFrame(columns=table.columns)    # Это избавит от ошибки, которая внизу выскакивает
                #df['Title'] = name
                #df['Year'] = year
                #df['Text'] = text
                #df['Artist'] = artist[0].upper() + artist[1:]
                #table = pd.concat([table, df], ignore_index=True)
        except IndexError:
            pass
        img_pages = items.find_all('a')
        for page in img_pages:
            href = page.get('href')
            if href != None and href != '#index':
                imgs.append(f"{url[:-10]}{href[2:-4]}.jpg")
                map = dict(zip(imgs, names))
    except AttributeError:
        pass
    return map, metadata

In [ ]:
def CGFA_result(CGFA_url):
    CGFA_artists = CGFA_Artists(CGFA_url)
    with ThreadPoolExecutor(8) as executor:
        img = {}
        metadata = pd.DataFrame(columns=['Title', 'Year', 'Type', 'Artist'])

        # Параллельно выполнять запросы и парсить HTML файлы
        results = [executor.submit(images_CGFA, url, metadata) for url in CGFA_artists]

        for result in results:
            try:
                imap, df = result.result()
                img.update(imap)
                metadata = pd.concat([metadata, df], ignore_index=True)

            except UnboundLocalError:
                continue
    img = pd.DataFrame.from_dict(img, orient='index').drop_duplicates().to_dict()[0]
    metadata.drop_duplicates(subset='Title', keep='first', inplace=True, ignore_index = True)
    #metadata.drop_duplicates()
    print("Scraping is finished!")
    return img, metadata

In [ ]:
def WGA_result(WGA_url):
    WGA_artists = WGA_Artists(WGA_url)

    with ThreadPoolExecutor(8) as executor:
        img = {}
        metadata = pd.DataFrame(columns=['Title', 'Year', 'Type', 'Artist'])

        # Параллельно выполнять запросы и парсить HTML файлы
        results = [executor.submit(images_WGA, url, metadata) for url in WGA_artists]

        for result in results:
            try:
                imap, df = result.result()
                img.update(imap)
                metadata = pd.concat([metadata, df], ignore_index=True)

            except UnboundLocalError:
                continue
    img = pd.DataFrame.from_dict(img, orient='index').drop_duplicates().to_dict()[0]
    #metadata.drop_duplicates(subset='Title', keep='first', inplace=True, ignore_index = True)
    #metadata.drop_duplicates()
    print("Scraping is finished!")
    return img, metadata

In [ ]:
img = {}
WGA_artists = WGA_Artists('https://www.wga.hu/support/mobile/artists.html')
metadata = pd.DataFrame(columns=['Title', 'Year', 'Type', 'Artist'])
for artist in WGA_artists:
    imap, df = images_WGA(artist, metadata)
    img.update(imap)
    #metadata = pd.concat([metadata, df], ignore_index=True)
    #img = pd.DataFrame.from_dict(img, orient='index').drop_duplicates().to_dict()[0]
    #metadata.drop_duplicates(subset='Title', keep='first', inplace=True, ignore_index = True)

In [ ]:
metadata

,Title,Year,Type,Artist
0,Venus and Adonis,1574-88,"Oil on canvas, 68 x 95 cm","AACHEN, Hans von"
1,Allegory,1598,"Oil on copper, 56 x 47 cm","AACHEN, Hans von"
2,"Allegory of Peace, Art and Abundance",1602,"Oil on canvas, 197 x 142 cm","AACHEN, Hans von"
3,"Jupiter, Antiope and Cupid",1595-98,"Oil on copper, 31 x 21 cm","AACHEN, Hans von"
4,"Pallas Athena, Venus and Juno",1593,"Oil on canvas, 54 x 67 cm","AACHEN, Hans von"
...,...,...,...,...
19874,Tulip vase with the arms of Willem III,1690s,Blue-painted faience height 98 cm,Delftware
19875,Large covered vase,1680s,"Blue-painted faience, height 96 cm",Delftware
19876,Octagonal vase,1680-85,"Polychrome faience, height 75 cm",Delftware
19877,Octagonal vase,1680-85,"Polychrome faience, height 75 cm",Delftware


In [ ]:
len(img)

19879

In [ ]:
img_copy = img

In [ ]:
WGA_data = metadata.copy()

In [ ]:
CGFA_url = 'http://www.sai.msu.su/cjackson/featured.htm'
CGFA_img, CGFA_metadata = CGFA_result(CGFA_url)

Scraping is finished!


In [ ]:
img_copy.update(CGFA_img)
metadata_final = pd.concat([WGA_data, CGFA_metadata], ignore_index=True)

In [ ]:
metadata_final

,Title,Year,Type,Artist
0,Venus and Adonis,1574-88,"Oil on canvas, 68 x 95 cm","AACHEN, Hans von"
1,Allegory,1598,"Oil on copper, 56 x 47 cm","AACHEN, Hans von"
2,"Allegory of Peace, Art and Abundance",1602,"Oil on canvas, 197 x 142 cm","AACHEN, Hans von"
3,"Jupiter, Antiope and Cupid",1595-98,"Oil on copper, 31 x 21 cm","AACHEN, Hans von"
4,"Pallas Athena, Venus and Juno",1593,"Oil on canvas, 54 x 67 cm","AACHEN, Hans von"
...,...,...,...,...
21011,Arrangement in Grey and Black Number 2: Port...,1872-73,Glasgow Art Gallery and Museum. 104KB,Whistler
21012,Nocturne in Black and Gold: The Falling Rocket,1875,Detroit Institute of Arts. 114KB,Whistler
21013,Arrangement in Brown and Black: Miss Rosa Co...,1876-78,Frick Collection,Whistler
21014,Arrangement in Black: The Lady in the Yellow...,1882-84,Philadelphia Museum of Art. 126KB,Whistler


In [ ]:
os.makedirs('images', exist_ok=True)
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Запуск задач для сохранения картинок
    futures = [executor.submit(download_image, url, filename) for url, filename in img_copy.items()]

    # Получение результатов задач
    for future in concurrent.futures.as_completed(futures):
        try:
            result = future.result()
        except Exception as e:
            print(f'Ошибка при сохранении картинки: {e}')

Выходные данные были обрезаны до нескольких последних строк (5000).
Image The Dream of Erwin von Steinbach.jpg saved successfully
Image Landscape with Wanderer.jpg saved successfully
Image Hercules Killing the Dragon in the Garden of the Hesperides.jpg saved successfully
Image The Baptism of Christ.jpg saved successfully
Image Death of Cleopatra.jpg saved successfully
Image Diptych.jpg saved successfully
Image Virgin and Child.jpg saved successfully
Image Portrait of a Man.jpg saved successfully
Image Lucretia.jpg saved successfully
Image The Flood.jpg saved successfully
Image Entry of Christ into Jerusalem.jpg saved successfully
Image Lochorst Triptych.jpg saved successfully
Image 12 Members of the Haarlem Brotherhood of Jerusalem Pilgrims.jpg saved successfully
Image Mary Magdalen.jpg saved successfully
Image Portrait of a Schoolboy.jpg saved successfully
Image Portrait of Joris van Egmond.jpg saved successfully
Image Portrait of a Man.jpg saved successfully
Image Portrait of a Man o

In [ ]:
round(metadata_final['Title'].nunique()*100/len(metadata_final['Title']), 1)

73.7

In [ ]:
metadata_final['Type'].unique()

array(['Oil on canvas, 68 x 95 cm', 'Oil on copper, 56 x 47 cm',
       'Oil on canvas, 197 x 142 cm', ...,
       'Detroit Institute of Arts. 114KB',
       'Philadelphia Museum of Art. 126KB', 'pastel on brown paper'],
      dtype=object)

In [ ]:
metadata_final.to_csv('metadata.csv', index=False)

In [ ]:
import zipfile

folder_path = '/content/images'
zip_path = '/content/images.zip'

zip_file = zipfile.ZipFile(zip_path, 'w')

for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        zip_file.write(file_path, os.path.basename(file_path))
zip_file.close()

print("Файлы успешно скачаны и сохранены в архиве images.zip")

Файлы успешно скачаны и сохранены в архиве images.zip
